In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.xarray 
import panel as pn
from thredds_crawler.crawl import Crawl
import datetime as dt
from scipy.interpolate import interp1d,interp2d

In [ ]:
hv.extension('bokeh')
pn.extension()

In [ ]:
def prep_mod(url):
    #opens model file from url as dataset
    mod = xr.open_dataset(url)
    
    #calculate height axis for the model, works for IFS
    mod['height'] = mod.zg - mod.zghalf[:, -1]
    
    #select relevant variable and height axis
    mod_ds = mod[['height','ta']]
    
    #flip the height axis and convert to dataframe
    mod_ds = mod_ds.reindex(level=list(reversed(mod_ds.level)))
    tot_df = mod_ds.to_dataframe()
    
    #resample with 1 hour frequency and remove first point in resulting dataframe
    # as it is at point 00:00:00, and the first model point when resampled should occur 
    #at 01:00:00
    df_res = tot_df.swaplevel().unstack().resample('1H').asfreq()
    df_res = df_res[1:]
    
    return df_res
    
def interpolate_mod(df_res,height):
    #create a dummy dataframe with the correct dimensions to hold the results
    time = df_res.index.values
    dummy = pd.DataFrame('NaN',height,time)

    #for each timestep - read for each height axis, interpolate to the new axis
    #then add to the dummy dataframe 
    for i in df_res.index.values:
        temp = df_res.loc[i].unstack().T
        height_model = temp['height']
        temperature = temp['ta']

        interpolated = interp1d(height_model,temperature,fill_value="extrapolate")(height)
        dummy[i] = interpolated
    
    
    return dummy

In [ ]:
def calc_err(mod,obs,hours):
#     #select relevant obs chunk from larger SOP obs file
#     st = mod.columns.values[0]
#     et = mod.columns.values[-1]
#     time_obs = ob[st:et]
    
    comb_func = lambda x,y: x-y
    err = dummy.combine(dummy_obs,comb_func)
    
    #convert to common time axis based on how many hours into the 3 
    #day forcast the point is
    err.columns = hours
    
    return err


def calculate(forcast_errors,hours):
    medians = []
    for hour in range(0,72):
        values_per_time = forcast_errors[:,hour,:]
        medians.append(np.median(values_per_time,axis=0))
    
    plottable= np.stack(medians).T
    
    return plottable


In [ ]:
def plot_err(plottable):
    ds = xr.DataArray(plottable,dims=['hieght','forcast length'])
    plot = ds.hvplot.contourf()
    return plot

In [ ]:
catalog = "https://thredds.met.no/thredds/catalog/alertness/YOPP_supersite/ifs-ecmwf/sodankyla/catalog.html"
c = Crawl(catalog, select=['.*20180[3]..00\.nc'])
urls_mod = []
for d in c.datasets:
    for s in d.services:
        if s.get("service").lower() == "opendap":
            urls_mod.append(s.get("url")+"?zg,zghalf,ta,time,level")
            
urls_mod = urls_mod[0:2]

errors = []
height = np.arange(0,8005,5)
hours = np.arange(1,73)


for url in urls_mod:
    df_res = prep_mod(url)
    dummy = interpolate_mod(df_res,height)
    
    dummy_obs = dummy.apply(lambda x: x-(x*0.25))
    err = calc_err(dummy,dummy_obs,hours)
    
    errors.append(err.T)
    
    
    
forcast_errors = np.stack(errors)

plottable = calculate(forcast_errors,hours)
plot_err(plottable)
